In [1]:
import ee
import ee.mapclient
import time
import datetime
import sys
!pip install geetools

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=8JZ6m7Z9TVLBhMQPbif3NxJy3KIf_txR199TC52ilCE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2AHmszpYfy-rT7epskfIdFRcqUbweTVUTXm170V5DVRz0hIROt1PZuQ

Successfully saved authorization token.


In [3]:
# Method for processing Sentinel-2 dataset
def process_sentinel(dataset, time_range, area):

    # remove clouds from images with masking
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_m = ee.Number(2).pow(10).int()
        cirrus_m = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_m).eq(0) and(
            band_qa.bitwiseAnd(cirrus_m).eq(0))
        return image.updateMask(mask).divide(10000)

    # produce filtered image using median
    filter_image = (ee.ImageCollection(dataset).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).
                         map(maskclouds))

    sentinel_median = filter_image.median()

    # col = ee.ImageCollection(dataset).
    #                      filterBounds(area).
    #                      filterDate(time_range[0], time_range[1]))
    # image = col.first();

    image_band = sentinel_median.select(['B4','B3','B2'])
    return sentinel_median

In [4]:
# Method for processing FLDAS dataset 
def process_FLDAS(dataset, timerange, coordinates, band):
    collection = (ee.ImageCollection(dataset)
            .filterDate(timerange[0], timerange[1])
            .filterBounds(coordinates)
            ).select(band);
    
    collection = collection.sort('CLOUD_COVER')

    image = collection.first().visualize(
          min= 0.0,
          max= 0.00005,
          opacity=1.0,
          palette= ["black", "blue", "purple", "cyan", "green", "yellow", "red"]
    )

    return image.clip(coordinates)

In [5]:
# Method for processing GLDAS dataset
def process_GLDAS(dataset, timerange, coordinates, band):
    collection = (ee.ImageCollection(dataset)
            .filterDate(timerange[0], timerange[1])
            .filterBounds(coordinates)
            ).select(band);
    
    collection = collection.sort('CLOUD_COVER')

    image = collection.first().visualize(
          min= 250.0,
          max= 300.0,
          palette= ['1303ff', '42fff6', 'f3ff40', 'ff5d0f'],
    )

    return image.clip(coordinates)

In [6]:
def download_data(datasets, timerange, coordinates, outputFolder, scale, report=False, testRun=True):
      
  print("Exporing Image to " + outputFolder + " Folder")

  for d_i,d in enumerate(datasets):
    
    print("\nProcessing " + d + "...")
    
    for c_i, c in enumerate(coordinates):

      if d == "COPERNICUS/S2":
        image = process_sentinel(d, timerange, c)
      elif d == "NASA/FLDAS/NOAH01/C/GL/M/V001":
        image = process_FLDAS(d, timerange, c, 'Evap_tavg')
      elif d == "NASA/GLDAS/V021/NOAH/G025/T3H":
        image = process_GLDAS(d, timerange, c, 'AvgSurfT_inst')
      else: 
        collection = (ee.ImageCollection(d)
                .filterDate(timerange[0], timerange[1])
                .filterBounds(c)
                )
        
        # print("Number of Images Found: " + str(collection.size().getInfo()))
        collection = collection.sort('CLOUD_COVER')

        image = collection.first().clip(c)

      if testRun:
        print("Dataset: " + str(d_i) + ", Coordinates: " + str(c_i))
      else:
        print("Sending img_"+ str(d_i) + "_" + str(c_i)+" to GEE...", end=" ")
        task = ee.batch.Export.image.toDrive(
            image=image,
            description="img_" + str(d_i) + "_" + str(c_i),
            folder="testing_ee",
            region=c,
            scale=30
        )
        task.start()
        print("\x1b[32mComplete\x1b[0m")

    print("Complete")

In [7]:
# Method to generate polygons from center points and grid sizes
def generate_polygons(coordinates, boxSize):
  polygons = []
  for c in coordinates:
    bw = boxSize[0]/2
    bh = boxSize[1]/2
    wc = c[0]
    hc = c[1]

    polygon = ee.Geometry.Polygon([
      [[wc-bw, hc-bh],
      [wc+bw, hc-bh],
      [wc+bw, hc+bh],
      [wc-bw, hc+bh],
      [wc-bw, hc-bh]
      ]
    ])

    polygons.append(polygon)
  return polygons

In [9]:
# datasets = ['COPERNICUS/S2', 'CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics', 'NASA/FLDAS/NOAH01/C/GL/M/V001', 'NASA/GLDAS/V021/NOAH/G025/T3H', 'JAXA/ALOS/PALSAR/YEARLY/FNF']
datasets = ['COPERNICUS/S2']

# Date range, datetime(year, month, day)
dates = [datetime.datetime(2010, 1, 1), datetime.datetime(2019, 5, 1)]

# Center coordinate and box size
centers = [[-80.0107, 40.4421], [-80.10123093348129,40.59933685509059]]
# centers = [[-80.0107, 40.4421]]
box = [0.02, 0.02]

download_data(datasets=datasets, timerange=dates, coordinates=generate_polygons(centers, box) , outputFolder='testing_ee', scale=1, testRun=False)

Exporing Image to testing_ee Folder

Processing COPERNICUS/S2...
Sending img_0_0 to GEE... Complete
Sending img_0_1 to GEE... Complete
Complete
